In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

file_path = "../healthcare_dataset.csv"

df=pd.read_csv(file_path)

df['Date of Admission']=pd.to_datetime(df['Date of Admission'])

df = df.sort_values("Date of Admission")
daily_admissions = df.groupby("Date of Admission").size().rename("admissions")

daily_admissions = daily_admissions.asfreq("D", fill_value=0)

print(daily_admissions.head())
print(daily_admissions.index.freq)

Date of Admission
2019-05-08    27
2019-05-09    38
2019-05-10    17
2019-05-11    25
2019-05-12    36
Freq: D, Name: admissions, dtype: int64
<Day>


In [5]:
test_days = 270

train = daily_admissions.iloc[:-test_days]
test  = daily_admissions.iloc[-test_days:]

print("Train:", train.index.min(), "to", train.index.max(), "len=", len(train))
print("Test :", test.index.min(),  "to", test.index.max(),  "len=", len(test))

Train: 2019-05-08 00:00:00 to 2023-08-11 00:00:00 len= 1557
Test : 2023-08-12 00:00:00 to 2024-05-07 00:00:00 len= 270


In [6]:
from statsmodels.tsa.statespace.sarimax import SARIMAX

sarima_model = SARIMAX(
    train,
    order=(1, 0, 1),
    seasonal_order=(1, 0, 1, 7),
    enforce_stationarity=False,
    enforce_invertibility=False
)

sarima_fit = sarima_model.fit(disp=False)
print(sarima_fit.summary())

                                     SARIMAX Results                                     
Dep. Variable:                        admissions   No. Observations:                 1557
Model:             SARIMAX(1, 0, 1)x(1, 0, 1, 7)   Log Likelihood               -5009.292
Date:                           Mon, 29 Dec 2025   AIC                          10028.584
Time:                                   13:10:17   BIC                          10055.308
Sample:                               05-08-2019   HQIC                         10038.524
                                    - 08-11-2023                                         
Covariance Type:                             opg                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
ar.L1         -0.1929      0.323     -0.597      0.550      -0.826       0.440
ma.L1          0.1797      0.324      0.554

In [7]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

sarima_forecast = sarima_fit.forecast(steps=len(test))
sarima_pred = sarima_forecast  # same thing, just naming

rmse = np.sqrt(mean_squared_error(test, sarima_pred))
mae  = mean_absolute_error(test, sarima_pred)
mape = np.mean(np.abs((test - sarima_pred) / test)) * 100
accuracy = 100 - mape

print("SARIMA performance on TEST set:")
print("RMSE:", rmse)
print("MAE :", mae)
print("MAPE:", mape, "%")
print("Accuracy (100 - MAPE):", accuracy, "%")

SARIMA performance on TEST set:
RMSE: 5.903071206187803
MAE : 4.73305524769943
MAPE: 16.633553922769295 %
Accuracy (100 - MAPE): 83.3664460772307 %


In [8]:
sarima_df = pd.DataFrame({
    "date": test.index,
    "actual": test.values,
    "sarima_pred": sarima_pred.values
})

sarima_df.to_csv("../270 days/predictions/sarima_predictions.csv", index=False)

print("SARIMA predictions saved successfully.")

SARIMA predictions saved successfully.


In [ ]:
#Rolling Series Model

series = daily_admissions.sort_index()

train_series = series.iloc[:-test_days]
test_series  = series.iloc[-test_days:]

history = train_series.copy()
rolling_preds = []

for t in range(len(test_series)):
    model = SARIMAX(
        history,
        order=(1,0,1),
        seasonal_order=(1,0,1,7),
        enforce_stationarity=False,
        enforce_invertibility=False
    )
    
    model_fit = model.fit(disp=False)
    
    y_pred = model_fit.forecast(steps=1)
    rolling_preds.append(y_pred.iloc[0])
    
    # add true value to history
    history = pd.concat([history, test_series.iloc[t:t+1]])

In [ ]:
sarima_rolling_pred = pd.Series(
    rolling_preds,
    index=test_series.index,
    name="sarima_rolling"
)

y_test = test_series

In [ ]:
rmse = np.sqrt(mean_squared_error(test_series, rolling_pred_series))
mae  = mean_absolute_error(test_series, rolling_pred_series)
mape = np.mean(np.abs((test_series - rolling_pred_series) / test_series)) * 100
accuracy = 100 - mape

print("Rolling SARIMA performance:")
print("RMSE:", rmse)
print("MAE :", mae)
print("MAPE:", mape)
print("Accuracy (100 - MAPE):", accuracy)

In [ ]:
#Plotting

plt.figure(figsize=(12,5))
plt.plot(test.index, test, label="Actual", color="black")
plt.plot(test.index, sarima_pred, label="SARIMA", alpha=0.7)
plt.plot(test.index, rolling_pred_series, label="Rolling SARIMA", alpha=0.7)

plt.title("Actual vs SARIMA vs Rolling SARIMA")
plt.xlabel("Date")
plt.ylabel("Admissions")
plt.legend()
plt.show()

In [ ]:
sarima_rolling_df = pd.DataFrame({
    "date": y_test.index,
    "actual": y_test.values,
    "sarima_rolling_pred": sarima_rolling_pred.values
})

sarima_rolling_df.to_csv("predictions/sarima_rolling_predictions.csv", index=False)

print("SARIMA rolling predictions saved successfully.")


In [ ]:
actual_df = pd.DataFrame({
    "date": y_test.index,
    "actual": y_test.values
})

actual_df.to_csv("predictions/actual.csv", index=False)